<a href="https://colab.research.google.com/github/TalHarel1998/RL_Project/blob/main/Copy_of_OpenAi_Gym_Atari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install dependancies, takes around 45 seconds

Rendering Dependancies



In [ ]:
#add " > /dev/null 2>&1" at the end of each command to ignore output
!pip install gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg

Dependancies

In [ ]:
!apt-get update
!pip install --upgrade setuptools
!pip install "gym[atari]"==0.9.5
!pip install opencv-python
!pip install torch
!pip install matplotlib

In [ ]:
# ERRORS CAN BE IGNORED! all the dependency conflicts are "won" by gym!
# (maybe because of the --force parameter ?)

!pip install --force "gym[atari]"==0.9.5

In [ ]:
# download, extract & import all available ROMs to gym 
!curl http://www.atarimania.com/roms/Roms.rar -o Roms.rar
!unrar x Roms.rar
!python -m atari_py.import_roms ./

In [ ]:
%cd /content
!rm -rf RL_Project
!git clone https://github.com/TalHarel1998/RL_Project.git
%cd /content/RL_Project/dqn

# Imports and Helper functions


In [ ]:
import gym
print(gym.__version__)  # make sure it says 0.9.5
from gym import logger as gymlogger
from gym import wrappers
import tensorflow as tf
import torch
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

# Classes From Skeleton

# Pong

In [ ]:
# # Get Atari games.
# benchmark = gym.benchmark_spec('Atari40M')
# task = benchmark.tasks[3]
# env = gym.make(task.env_id)

env = gym.make('PongNoFrameskip-v4')  # this command should succeeds

from utils.atari_wrapper import ProcessFrame84
env = ProcessFrame84(env)

## simulating random exploation

In [ ]:
import random
from utils.replay_buffer import ReplayBuffer

replay_buffer = ReplayBuffer(100, 7)

last_obs = env.reset()


for i in range(10):

  stored_idx = replay_buffer.store_frame(last_obs)

  # action = random.randrange(env.action_space.n)

  action = 

  obs, reward, done, info = env.step(action)

  print(obs.shape)

  encoded_obs = replay_buffer.encode_recent_observation()

  replay_buffer.store_effect(stored_idx, action, reward, done)

  last_obs = obs

## attempting to run the model

In [ ]:
import numpy as np
import torch
from dqn_model import DQN

obs_batch = torch.from_numpy(np.random.rand(10, 1, 84, 84)).type(torch.float32)
print(obs_batch.shape)

model = DQN(in_channels = 1)
res = model(obs_batch)

print(res.data.max(1)[0])
print([max(res.data[i]) for i in range(0,10,2)])


In [ ]:
import numpy as np
import torch.autograd as autograd
from dqn_model import DQN

class Variable(autograd.Variable):
    def __init__(self, data, *args, **kwargs):
        if torch.cuda.is_available():
            data = data.cuda()
        super(Variable, self).__init__(data, *args, **kwargs)

def select_epilson_greedy_action(model, obs):
        obs = torch.from_numpy(obs).type(torch.float32).unsqueeze(0) / 255.0
        # with torch.no_grad() variable is only used in inference mode, i.e. don’t save the history
        with torch.no_grad():
            return model(Variable(obs, volatile=True)).data.max(1)[1].cpu()
        
model = DQN(in_channels = 1)

last_obs = env.reset()

obs, reward, done, info = env.step(0)

select_epilson_greedy_action(model, np.moveaxis(obs, -1, 0))

In [ ]:
import numpy as np
import torch
import torch.optim as optim

obs_batch = torch.from_numpy(np.random.rand(10, 1, 84, 84)).type(torch.float32)
print(obs_batch.shape)

model = DQN(in_channels = 1)

loss = torch.nn.MSELoss

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
res = sum(model(obs_batch).max(1)[0])

optimizer.zero_grad()

print(res.backward())

## running main

In [ ]:
!python main.py

### code that runs before t==1 (main.py + imports + initializations)


In [ ]:
# ------------------------ content of main.py ------------------------

import gym
import torch.optim as optim

from dqn_model import DQN
from dqn_learn import OptimizerSpec, dqn_learing
from utils.gym import get_env, get_wrapper_by_name
from utils.schedule import LinearSchedule

BATCH_SIZE = 32
GAMMA = 0.99
REPLAY_BUFFER_SIZE = 1000000
LEARNING_STARTS = 50000
LEARNING_FREQ = 4
FRAME_HISTORY_LEN = 1           #TODO: change back to 4 !
TARGER_UPDATE_FREQ = 10000
LEARNING_RATE = 0.00025
ALPHA = 0.95
EPS = 0.01



# Get Atari games.
benchmark = gym.benchmark_spec('Atari40M')

# Change the index to select a different game.
task = benchmark.tasks[3]

# Run training
seed = 0 # Use a seed of zero (you may want to randomize the seed!)
env = get_env(task, seed)

num_timesteps = task.max_timesteps

# ---------------------------------- content of main ----------------------------------

def stopping_criterion(env):
    # notice that here t is the number of steps of the wrapped env,
    # which is different from the number of steps in the underlying env
    return get_wrapper_by_name(env, "Monitor").get_total_steps() >= num_timesteps

optimizer_spec = OptimizerSpec(
    constructor=optim.RMSprop,
    kwargs=dict(lr=LEARNING_RATE, alpha=ALPHA, eps=EPS),
)

exploration_schedule = LinearSchedule(1000000, 0.1)

q_func=DQN
optimizer_spec=optimizer_spec
exploration=exploration_schedule
stopping_criterion=stopping_criterion
replay_buffer_size=REPLAY_BUFFER_SIZE
batch_size=BATCH_SIZE
gamma=GAMMA
learning_starts=LEARNING_STARTS
learning_freq=LEARNING_FREQ
frame_history_len=FRAME_HISTORY_LEN
target_update_freq=TARGER_UPDATE_FREQ

# ---------------------------------- imports of dqn_learn ----------------------------------

import sys
import pickle
import numpy as np
from collections import namedtuple
from itertools import count
import random
import gym.spaces

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd

from utils.replay_buffer import ReplayBuffer
from utils.gym import get_wrapper_by_name

# ---------------------------------- init content of dqn_learn ----------------------------------

USE_CUDA = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

if len(env.observation_space.shape) == 1:
    # This means we are running on low-dimensional observations (e.g. RAM)
    input_arg = env.observation_space.shape[0]
else:
    img_h, img_w, img_c = env.observation_space.shape
    input_arg = frame_history_len * img_c
num_actions = env.action_space.n

# Construct an epilson greedy policy with given exploration schedule
def select_epilson_greedy_action(model, obs, t):
    sample = random.random()
    eps_threshold = exploration.value(t)
    if sample > eps_threshold:
        obs = torch.from_numpy(obs).type(dtype).unsqueeze(0) / 255.0
        # with torch.no_grad() variable is only used in inference mode, i.e. don’t save the history
        with torch.no_grad():
            return model(Variable(obs)).data.max(1)[1].cpu()
    else:
        return torch.IntTensor([[random.randrange(num_actions)]])


### exploration&training loop

In [ ]:


# Initialize target q function and q function, i.e. build the model.
######

# YOUR CODE HERE
Q        = q_func(input_arg, num_actions)
target_Q = q_func(input_arg, num_actions)

######

# Construct Q network optimizer function
optimizer = optimizer_spec.constructor(Q.parameters(), **optimizer_spec.kwargs)

# Construct the replay buffer
replay_buffer = ReplayBuffer(replay_buffer_size, frame_history_len)


###############
# RUN ENV     #
###############
num_param_updates = 0
mean_episode_reward = -float('nan')
best_mean_episode_reward = -float('inf')
last_obs = env.reset()
LOG_EVERY_N_STEPS = 10000

for t in count():
    ### 1. Check stopping criterion
    if stopping_criterion is not None and stopping_criterion(env):
        break

    # YOUR CODE HERE

    # Steps the environment forward one step
    action = select_epilson_greedy_action(Q, np.moveaxis(last_obs, -1, 0), t)
    obs, reward, done, info = env.step(action)

    # Stors and updates replay_buffer's variables due the latest observation
    stored_idx  = replay_buffer.store_frame(last_obs)
    # TODO: do we need the encoded_obs? maybe we'll have to use it during the learning process / 
    if t > 0:
        encoded_obs = replay_buffer.encode_recent_observation()
    replay_buffer.store_effect(stored_idx, action, reward, done)

    if done:
        last_obs = env.reset()
    else:
        last_obs = obs

    #####


    if (t > learning_starts and
            t % learning_freq == 0 and
            replay_buffer.can_sample(batch_size)):
        
        print(f"breaking at t={t}")
        break

In [ ]:
obs_batch, act_batch, rew_batch, next_obs_batch, done_mask = replay_buffer.sample(batch_size)
        
obs_batch, act_batch, rew_batch, next_obs_batch, done_mask = \
    torch.from_numpy(obs_batch).type(dtype) / 225.0, \
    torch.from_numpy(act_batch).type(dtype), \
    torch.from_numpy(rew_batch).type(dtype), \
    torch.from_numpy(next_obs_batch).type(dtype) / 255.0, \
    torch.from_numpy(done_mask)

obs_batch, next_obs_batch = Variable(obs_batch), Variable(next_obs_batch)

# compute bellman error
Q_next = Q(next_obs_batch).data.max(1)[0]
Q_next = Q_next * done_mask # if obs.done=True, Q_next is obviously zero
Q_forward = Q(obs_batch) 

expanded_act_batch = act_batch.unsqueeze(1).expand(-1, Q_forward.size(1)).type(torch.int64)
sliced_Q_forward = torch.gather(Q_forward, 1, expanded_act_batch)[:,0]

sliced_Q_forward.shape

bellman_error = rew_batch + gamma * Q_next - torch.detach(sliced_Q_forward).numpy()
bellman_error = np.clip(bellman_error, -1, 1) * -1
bellman_error = torch.sum(bellman_error)

# # make an optimization step
# optimizer.zero_grad()
# bellman_error.backward()
# optimizer.step()

# if (num_param_updates % target_update_freq) == 0:
#     target_Q.load_state_dict(Q.state_dict())

# num_param_updates += 1

In [ ]:
optimizer.zero_grad()
bellman_error.backward()
optimizer.step()